In [ ]:
import pandas as pd

In [ ]:
cic = pd.read_csv('cicids2017_preprocessed.csv')

In [ ]:
ton = pd.read_csv('type_before_pre_ton.csv')

In [ ]:
del ton['Unnamed: 0']

In [ ]:
# 공통 feature 이름 매핑

In [ ]:
ton.rename(columns = {'duration':'Flow Duration','proto':'Protocol','src_bytes':'Fwd Packet Length Mean','dst_bytes':'Bwd Packet Length Mean','src_pkts':'Total Fwd Packets','src_ip_bytes':'Fwd Packets/s','dst_pkts':'Total Bwd Packets','dst_ip_bytes':'Bwd Packets/s','type':'Label'},inplace=True)
cic.rename(columns = {'Total Backward Packets':'Total Bwd Packets'},inplace=True)

In [ ]:
# 필요 없는 feature 제거 (타임스탬프, 포트 등)

In [ ]:
ton = ton.drop(['label', 'ts', 'src_port', 'dst_port'], axis= 'columns')

In [ ]:
cic = cic.drop(['Source IP', 'Source Port', 'Destination IP', 'Destination Port','Timestamp', 'External IP], axis= 'columns')

In [ ]:
# Ton-IoT 데이터셋 크기에 맞게 Sampling

In [ ]:
cic_s = cic.sample(n=461043, replace=False, axis=0)

In [ ]:
# Ton-IoT, CICIDS2017 데이터셋 Merge

In [ ]:
frames = [cic_s, ton]
df = pd.concat(frames)

In [ ]:
# 결측값을 -1로 변환하고 결측값 존재하는지 확인

In [ ]:
df.fillna(-1,inplace=True)

In [ ]:
df.isna().sum()

In [ ]:
# 공격 타입 통일

In [ ]:
df = df.replace('normal','Benign')
df = df.replace('scanning','PortScan')
df = df.replace('ddos','DDoS')
df = df.replace('dos','DoS')
df = df.replace('injection','Injection')
df = df.replace('password','Password')
df = df.replace('xss','XSS')
df = df.replace('ransomware','Ransomware')
df = df.replace('backdoor','Backdoor')
df = df.replace('mitm','MITM')

df = df.replace('BENIGN','Benign')
df = df.replace('DoS GoldenEye','DoS')
df = df.replace('DoS slowloris','DoS')
df = df.replace('DoS Slowhttptest','DoS')
df = df.replace('DoS Hulk','DoS')

In [ ]:
# Feature Importance 계산

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random

X = df[['Flow ID', 'Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Bwd Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio', 'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate', 'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'Init_Win_bytes_forward', 'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward', 'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min', 'service', 'conn_state', 'missed_bytes', 'dns_qclass', 'dns_qtype', 'dns_rcode', 'dns_AA', 'dns_RD', 'dns_RA', 'dns_rejected', 'ssl_version', 'ssl_cipher', 'ssl_resumed', 'ssl_established', 'ssl_subject', 'ssl_issuer', 'http_trans_depth', 'http_method', 'http_version', 'http_request_body_len', 'http_response_body_len', 'http_status_code', 'http_orig_mime_types', 'http_resp_mime_types', 'weird_name', 'weird_addl', 'weird_notice']]
y = df['Label']

labels = X.columns

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X, y)

importance = model.feature_importances_
std = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)
indices = np.argsort(importance)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("{}. {} ({:.5f})".format(f + 1, X.columns[indices][f], importance[indices[f]]))